In [ ]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
df = pd.read_csv('smartphones.csv')
df.head(n=5)

In [ ]:
df.info()

In [ ]:
sum(df.duplicated())

In [ ]:
df.nunique()

In [ ]:
df_mean_pop = df.groupby( ['brand_name'])['price'].mean().to_frame(name = 'price')#.reset_index()
df_mean_pop.sort_values(['price'], ascending=False).head(n=10)

In [ ]:
df['brand_name'].value_counts().plot(kind='bar', title='Contagem de modelos por marca', alpha=.7)
plt.xlabel('brand_name', fontsize=10)
plt.ylabel('model', fontsize=10);

In [ ]:
ram = df.groupby( ['ram_capacity'])['brand_name'].count().to_frame(name = 'brand_name')#.reset_index()
ram.sort_values(['brand_name'], ascending=False).head(n=10)